In [1]:
# libraries import
from keras.models import Sequential
from keras import layers
from keras.models import Model
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# file import
import data_cleaner as dc
import model_helper as mh

Using TensorFlow backend.


In [7]:
class Model:
    def __init__(self, neuron=100, min_df = 0):
        self.neuron = neuron
        self.df = dc.clean_item_data(0)
        return_obj = dc.cleanup_categoryid(self.df)
        self.df = return_obj[0]
        self.category_dict = return_obj[1]

        # vectorize training input data
        _X_train, _X_valid, _X_test, self.Y_train, self.Y_valid, self.Y_test = dc.data_split(self.df, 0.65, 0.15, 0.20)
        if min_df != 0:
            self.vectorizer = CountVectorizer(encoding='latin1', min_df = min_df) # Allow different options (min_df, encoding)
        else:
            self.vectorizer = CountVectorizer(encoding='latin1') # Allow different options (min_df, encoding)

        # convert pandas dataframes to list of strings
        x_train_list = []
        x_test_list = []
        x_valid_list = []
        for _, row in _X_train.iterrows():
            x_train_list.append(row[0])
        for _, row in _X_test.iterrows():
            x_test_list.append(row[0])
        for _, row in _X_valid.iterrows():
            x_valid_list.append(row[0])

        self.vectorizer.fit(x_train_list)
        self.X_train = self.vectorizer.transform(x_train_list)
        self.X_test = self.vectorizer.transform(x_test_list)
        self.X_valid = self.vectorizer.transform(x_valid_list)
    
    def train_model(self):
        # Neural Network
        input_dim = self.X_train.shape[1] # Number of features
        output_dim = self.df['categoryId'].nunique()
        model = Sequential()
        model.add(layers.Dense(self.neuron, input_dim=input_dim, activation='relu', use_bias=False))
        model.add(layers.Dropout(rate=0.6))
        model.add(layers.Dropout(rate=0.6))
        model.add(layers.Dense(output_dim, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        history = model.fit(self.X_train, self.Y_train,
                            epochs=4,
                            verbose=1,
                            validation_data=(self.X_valid, self.Y_valid),
                            batch_size=10)
        #print(model.summary())

        loss, self.train_accuracy = model.evaluate(self.X_train, self.Y_train, verbose=False)
        loss, self.test_accuracy = model.evaluate(self.X_test, self.Y_test, verbose=False)
        self.model = model
        
    def get_accuracy(self):
        return (round(self.train_accuracy, 4), round(self.test_accuracy, 4))
    
    
    def get_category(self,s):
        result = False
        s_arr = np.array([s])
        vector = self.vectorizer.transform(s_arr)
        prediction = self.model.predict_classes(vector)[0]
        
        for key in category_dict:
            if category_dict[key] == prediction:
                result = prediction
        if result:
            return result
        else:
            raise Exception('Fatal Error: Invalid model prediction')
    
    def stat(self):
        pass

In [3]:
model = Model()

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
model.train_model()

Train on 7066 samples, validate on 1663 samples
Epoch 1/4
7066/7066 [==============================] - 15s 2ms/step - loss: 3.6090 - acc: 0.2104 - val_loss: 2.5452 - val_acc: 0.7192
Epoch 2/4
7066/7066 [==============================] - 14s 2ms/step - loss: 2.3677 - acc: 0.4424 - val_loss: 1.4934 - val_acc: 0.7895
Epoch 3/4
7066/7066 [==============================] - 14s 2ms/step - loss: 1.8360 - acc: 0.5401 - val_loss: 1.0934 - val_acc: 0.8184
Epoch 4/4
7066/7066 [==============================] - 15s 2ms/step - loss: 1.5427 - acc: 0.5957 - val_loss: 0.9059 - val_acc: 0.8268


In [5]:
model.get_accuracy()

(0.9169, 0.8489)

In [6]:
model.get_category('toothbrush')

IndexError: invalid index to scalar variable.